# Non-Normal Confidence Intervals in R

During the lecture videos, we took a look at how we would construct a confidence interval for the rate parameter $\lambda$ of an exponential distribution. In particular, we determined how to construct confidence intervals using two different statistics: the mean $\bar{X}$ and the minimum value.

Let's use R to simulate an exponential distribution, and see if our confidence intervals actually work the way we think they do. 

Start by generating 15 samples from an $Exp(5)$ distribution. To do this, use the `rexp()` function with parameter `rate=5`.

In [4]:
#rate=5
#set seed for reproducibility
set.seed(0)

#senerate 15 samples from Exp(5)
sample_exp = rexp(n = 15, rate = 5)

# Display the sample
print(sample_exp)

 [1] 0.03680732 0.02914135 0.02795905 0.08721373 0.57899371 0.24591241
 [7] 0.10793657 0.19131350 0.02940920 0.27814703 0.15240597 0.24752071
[13] 0.88478684 0.21090863 0.20704879


That's a single sample. If we want to look at multiple confidence intervals, we're going to want multiple samples. Let's look at 100 samples in total.

We can calculate multiple samples at once, and store them into a matrix using the `matrix()` function. For our matrix, each sample will be a seperate row, and there will be 15 columns because we're generating 10 data points per sample, so our final function call will look like `matrix(rexp(15*100, rate=5), ncol=10)`. Use `set.seed(0)` to make sure you get the same values each time.

In [5]:


# Generate 100 samples of 10 data points each from Exp(5)
data = matrix(rexp(100 * 10, rate = 5), ncol = 10, byrow = TRUE)

# View the first few rows of the matrix
head(data)

0.37520703,0.13094933,0.0673867,0.11769594,0.47290305,0.128378518,0.05882408,0.1131731,0.02121452,0.01188783
0.11574249,0.79178657,0.2346624,0.19936259,0.28705707,0.007453705,0.06480203,0.2640936,0.04070207,0.20454518
0.06034819,0.14504286,0.1503085,0.04700549,0.21597623,0.205649381,0.25845233,0.2506211,0.11092828,0.06025660
0.25862493,0.19891116,0.1028349,0.40156648,0.08444849,0.435754514,0.64355780,0.1115659,0.11892353,0.19547916
0.04197332,0.06188957,0.2211873,0.15483755,0.01793482,0.221635331,0.04945285,0.3143974,0.96656255,0.08622643
0.54607786,0.22736628,0.1626736,0.16740130,0.35695308,0.462494325,0.58197745,0.0571182,0.07775735,0.01041109


##### Let's start by looking at the confidence interval that we made with the mean. From the lectures, we eventually got the equation

$$ \dfrac{\chi^2_{0.025, 2n}}{2n\bar{X}} \le \lambda \le \dfrac{\chi^2_{0.975, 2n}}{2n\bar{X}} $$

If you don't remember how we got these equations, please go over your notes to make sure you understand. For now, we can use these equations to create our confidence intervals. For the $\chi^2$ values, we can use the `qchisq()` function with 15*2 degrees of freedom. For the lower bound, want `qchisq(0.025, 30)` and the upper bound we want `qchisq(0.975, 30)`.

We want to calculate the upper and lower bounds for each of our samples. We could use a for loop for this, but it would be more efficient to use the `apply()` function. To do that, we will need to define our calculations as functions. Fill in the functions below with the necesary calculations for each bound of the confidence interval.

In [6]:
#lower bound function for confidence interval using the mean
exp.conf.int.lower = function(sample) {
  n = length(sample)
  xbar = mean(sample)
  lower = qchisq(0.025, df = 2 * n) / (2 * n * xbar)
  return(lower)
}

# Upper bound function for confidence interval using the mean
exp.conf.int.upper = function(sample) {
  n = length(sample)
  xbar = mean(sample)
  upper = qchisq(0.975, df = 2 * n) / (2 * n * xbar)
  return(upper)
}

Now we have our two functions defined, we can use them to calculate the upper and lower bounds for the entire matrix. We will define the apply function as `apply(data, 1, exp.conf.int.lower)`.
* `data` is the variable containing your matrix of samples. It is the data that the function will be applied to.
* `1` means we are applying our function to each row of the data. This is the "margin" that we apply the function along. If we wanted to apply the function to each column, we would input `2`.
* `exp.conf.int.lower` is the function that we will apply to the data.

Use this function for both the `exp.conf.int.lower` and `exp.conf.int.upper` functions to get vectors of the numeric upper and lower bounds for each sample.

In [7]:
#Apply the lower and upper bound functions to each sample (row)
lower.bounds = apply(data, 1, exp.conf.int.lower)
upper.bounds = apply(data, 1, exp.conf.int.upper)

#display the first few bounds as a check
head(lower.bounds)
head(upper.bounds)

[1] 3.202006 2.169655 3.187175 1.879316 2.244930 1.809423

[1] 11.407969  7.729954 11.355130  6.695546  7.998140  6.446535

How many of these confidence intervals contain the true parameter? Recall that you generated your data with a know rate $\lambda=5$.

We can perform an arithmetic comparison between a number and a vector in R, and R is smart enough to perform that operation between that number and each element of that vector. That's a complicated way of saying we can do `lower.bound < 5` and `5 < upper.bound`, and get vectors of boolean values for the "truth value" of each element's comparison. In total, want the condition `lower.bound < 5 < upper.bound`, so we can use a logical "and" to combine the two boolean vectors.

Combining all that, we get `(lower.bound < 5) & (5 < upper.bound)`. This gives us a vector of boolean values, where each boolean is whether or not $\lambda=5$ was within the confidence interval. To find the total number, we can calculate the sum of this vector. Does this value match the original confidence?

In [9]:
#count how many confidence intervals contain the true rate λ = 5
contains_lambda = (lower.bounds < 5) & (5 < upper.bounds)

# Sum up the number of TRUEs to get total hits
num_containing_lambda = sum(contains_lambda)

# Print result
num_containing_lambda

[1] 97

Great! That's our first confidence interval done! Now let's move to the second confidence interval, which used the minimum value of the sample. Recall from the lectures that the 95% confidence interval was defined as:

$$ 0 \le \lambda \le \dfrac{-ln(0.05)}{nY_n} $$

where $Y_n = min(X_1,X_2, \dots, X_n)$.

Let's repeat the process above of defining a function for the bounds. Because we know that the lower bound is 0, we don't have to worry about defining anything for that. Complete the function below to solve for the upper bound of the confidence interval.

Note: We defined the lower bound at 0, for simplicity. This doesn't have to be the case. We could have a 95% confidence interval that had a lower bound at a different value, which would cause the upper bound to shift higher, as the confidence interval would have to continue containing 95% of the area.

In [13]:
ci.upper.bound.min = function(sample){
  n = length(sample)
  y_n = min(sample)
  upper = -log(0.05) / (n * y_n)
  return(upper)
}


Again, use the `apply()` function to calculate the upper bound for the confidence intervals of each sample's data. Then compute whether `0 <= 5 <= upper.bound` for each sample. How many times the confidence interval contained the true rate? Does it match the 95% confidence level of the interval?

In [14]:
#apply the min-based CI upper bound function to each sample
upper.bounds.min = apply(data, 1, ci.upper.bound.min)

# Since lower bound is always 0, check if 5 is within [0, upper bound]
contains.lambda.min = 5 <= upper.bounds.min

# Count how many intervals contain the true lambda = 5
num.contains.lambda.min = sum(contains.lambda.min)

# Output the result
num.contains.lambda.min

[1] 97